In [ ]:
# 2차 다항 회귀 함수를 이용한 거리 보정 데이터 수집 코드

import cv2
from ultralytics import YOLO

model = YOLO('/home/choigh/WS/IP_OB/Installation_point/yolo_data/runs/detect/train3/weights/best.pt')

cap = cv2.VideoCapture(0)

# 수집된 데이터 저장용 리스트
bbox_width_list = []
real_distance_list = []

print("[INFO] 's' 키를 눌러 bbox_w 측정, 실측 거리 입력")
print("[INFO] 'q' 키를 누르면 종료됩니다")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]

    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        bbox_w = x2 - x1

        # 시각화
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} w={bbox_w}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        break  # 첫 번째 물체만 사용

    cv2.imshow('Measurement Mode', frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):  # save
        print(f"\n[BBOX WIDTH] → {bbox_w}px")
        cm = input("[INPUT] 실제 거리(cm)를 입력하세요: ")
        try:
            cm = float(cm)
            bbox_width_list.append(bbox_w)
            real_distance_list.append(cm)
            print(f"[SAVED] bbox_w: {bbox_w}, real_distance: {cm}")
        except:
            print("[ERROR] 숫자로 입력하세요!")

    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 결과 출력
print("\n[RESULT]")
print("bbox_width_list =", bbox_width_list)
print("real_distance_list =", real_distance_list)
